In [1]:
%python
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
mlflow_experiment_id = 866112
import mlflow
import mlflow.spark

In [2]:
## IMPORT DATA FOR EDA
vg_data = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/vgsales.csv')
vg_data.createOrReplaceTempView("vgsales")

In [3]:
%sql
SELECT Genre, count(1) AS count FROM vgsales GROUP BY Genre ORDER BY count DESC

Genre,count
Action,3316
Sports,2346
Misc,1739
Role-Playing,1488
Shooter,1310
Adventure,1286
Racing,1249
Platform,886
Simulation,867
Fighting,848


In [4]:
%sql
SELECT Genre, sum(Global_Sales) AS sum FROM vgsales GROUP BY Genre ORDER BY sum DESC

Genre,sum
Action,1751.1799999999691
Sports,1330.929999999988
Shooter,1037.3699999999901
Role-Playing,927.3699999999941
Platform,831.3699999999974
Misc,809.9599999999936
Racing,732.0399999999955
Fighting,448.90999999999923
Simulation,392.19999999999783
Puzzle,244.9500000000005


In [5]:
%sql
SELECT Platform, sum(Global_Sales) AS sum FROM vgsales GROUP BY Platform ORDER BY sum DESC

Platform,sum
PS2,1255.6399999999871
X360,979.9599999999996
PS3,957.8399999999987
Wii,926.7099999999971
DS,822.4899999999874
PS,730.659999999997
GBA,318.499999999998
PSP,296.2799999999948
PS4,278.0999999999994
PC,258.81999999999846


In [6]:
%sql
SELECT Year, count(1) FROM vgsales GROUP BY Year ORDER BY Year

Year,count(1)
1980,9
1981,46
1982,36
1983,17
1984,14
1985,14
1986,21
1987,16
1988,15
1989,17


In [7]:
%python
## LOAD DATA INTO DATAFRAME
df=pd.read_csv('/dbfs/FileStore/tables/vgsales.csv')

In [8]:
## VIEW AND CLEAN DATA
#Drop empty Publisher rows and fill empty Year rows with median
df.dropna(subset=['Publisher'], axis = 0, inplace=True)
df['Year'].fillna(df['Year'].median(), inplace=True)

In [9]:
## PROCESSING AND FEATURE ENGINEERING
from sklearn.preprocessing import LabelEncoder
# Drop irrelevant data
df.drop('Name', axis=1, inplace=True)
df.drop('Rank', axis=1, inplace=True)
# Label encode categorical data
le = LabelEncoder()
df['Publisher'] = le.fit_transform(df['Publisher'])
df['Platform'] = le.fit_transform(df['Platform'])
df['Genre'] = le.fit_transform(df['Genre'])
# convert numerical data to float 16
df['Platform'] = df['Platform'].astype('float16')
df['Year'] = df['Year'].astype('float16')
df['Genre'] = df['Genre'].astype('float16')
df['Publisher'] = df['Publisher'].astype('float16')
df['NA_Sales'] = df['NA_Sales'].astype('float16')
df['EU_Sales'] = df['EU_Sales'].astype('float16')
df['JP_Sales'] = df['JP_Sales'].astype('float16')
df['Other_Sales'] = df['Other_Sales'].astype('float16')
df['Global_Sales'] = df['Global_Sales'].astype('float16')

In [10]:
## DEFINE TARGET AND FEATURES
y = np.array(df['Global_Sales'])
X = df.drop('Global_Sales', axis = 1)
feature_list = list(df.columns)
X = np.array(X)

['Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']

In [11]:
## SPLIT DATA INTO TEST AND TRAIN
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

In [12]:
## Run Models
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
    
def model_rf():
    model = RandomForestRegressor(
    n_estimators=100,
    max_depth= 10,
    random_state=2020,
    n_jobs=-1)
    return model

In [13]:
import pickle
import gc
def train_model (X_train, y_train):
    kf = KFold(5)
    for i, (tr_idx, vl_idx) in enumerate(kf.split(X_train, y_train)):
        print('FOLD {} \n'.format(i))
        X_tr, y_tr = X_train[tr_idx], y_train[tr_idx]
        X_vl, y_vl = X_train[vl_idx], y_train[vl_idx]
        model = model_rf()
        model.fit(X_tr, y_tr)
        with open('rf_model_{}.pkl'.format(i), 'wb') as handle:
            pickle.dump(model, handle)
        handle.close()
        del model, X_tr, X_vl
        gc.collect()
        

In [14]:
train_model(X_train=X_train, y_train=y_train)

FOLD 0 

FOLD 1 

FOLD 2 

FOLD 3 

FOLD 4

In [15]:
## EVALUATE MODELS
MAEscore = []
accuracyscore = []
predictions = []
for i in range(5):
    infile = open('rf_model_{}.pkl'.format(i), 'rb')
    model = pickle.load(infile)
    infile.close()
    prediction = model.predict(X_test)
    predictions.append(prediction)
    errors = abs(predictions - y_test)
    meanerrors = round(np.mean(errors), 2)
    MAEscore.append(meanerrors)
    mape = np.mean(100 * (errors/y_test))
    accuracy = 100 - mape
    accuracyscore.append(accuracy)

In [16]:
## VISUALIZE FEATURE IMPORTANCES
importances = list(model.feature_importances_)
feature_importances = [(feature, round(importances, 2)) for feature, importances in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: NA_Sales Importance: 0.83
Variable: EU_Sales Importance: 0.12
Variable: Other_Sales Importance: 0.03
Variable: JP_Sales Importance: 0.02
Variable: Platform Importance: 0.0
Variable: Year Importance: 0.0
Variable: Genre Importance: 0.0
Variable: Publisher Importance: 0.0

In [17]:
# Plot
plt.style.use('fivethirtyeight')
x_values = list(range(len(importances)))
plt.bar(x_values, importances, orientation='vertical')
plt.xticks(x_values, feature_list, rotation='vertical')
plt.ylabel('Importance')
plt.xlabel('Variable')
plt.title('Variable Importances')

In [18]:
## VISUSALIZE PREDICTIONS
import datetime
#years = X[:, feature_list.index('Year')]
years_test = X_test[:, feature_list.index('Year')]
true_data = pd.DataFrame(data = {'Year': years_test, 'actual': y_test})
true_data = true_data.groupby(["Year"])["actual"].sum()
predictions = np.mean(predictions, axis=0)
predictions_data = pd.DataFrame(data = {'Year': years_test, 'prediction': predictions})
predictions_data = predictions_data.groupby(["Year"])["prediction"].sum()
plt.plot(true_data, label = 'actual')
plt.plot(predictions_data, label = 'prediction')
plt.legend()
plt.xlabel ('Year')
plt.ylabel ('Global Sales (in millions)')
plt.title('Actual and Predicted Sales')